In [1]:
%cd /Users/mchladek/Dev/article-tagging/lib/tagnews/

from utils.utils import load_data_subset
import pandas as pd
import numpy as np

/Users/mchladek/Dev/article-tagging/lib/tagnews


LOAD DATA

In [2]:
%cd /Users/mchladek/Dev/chicago-justice-data/data_subset

newssource_df, articles_df, categories_df, trainedcategoryrelevance_df, trainedcoding_df, usercoding_df,  usercoding_categories_df, trainedlocation_df = load_data_subset()

/Users/mchladek/Dev/chicago-justice-data/data_subset
news sources loaded. size: (38, 4)
articles loaded. size: (50000, 10)
categories loaded. size: (40, 6)
trainedcategoryrelevance loaded. size: (53970, 4)
trainedlocation loaded. size: (12396, 8)
trainedcoding loaded. size: (25000, 8)
usercoding loaded. size: (25000, 7)
usercoding_categories loaded. size: (13191, 3)


GET CRIME CATEGORY IDs

In [3]:
domv_cat = int(categories_df.loc[categories_df['abbreviation']=='DOMV', 'id'].squeeze())

In [4]:
sexa_cat = int(categories_df.loc[categories_df['abbreviation']=='SEXA', 'id'].squeeze())

USERCODING

Get user codings with relevant crime categories

In [5]:
# Can drop 'id' column from categories because not needed
rel_usercoded_df = pd.merge(left=usercoding_df,
              right=usercoding_categories_df.drop(columns=['id']),
              left_on="id",
              right_on="usercoding_id")
rel_usercoded_df = rel_usercoded_df[(rel_usercoded_df["relevant"] == "t") &
                                    ((rel_usercoded_df["category_id"] == domv_cat) |
                                     (rel_usercoded_df["category_id"] == sexa_cat))]

In [6]:
rel_usercoded_df.loc[rel_usercoded_df['category_id'] == domv_cat, 'category_id'] = 'DOMV'
rel_usercoded_df.loc[rel_usercoded_df['category_id'] == sexa_cat, 'category_id'] = 'SEXA'
rel_usercoded_df = rel_usercoded_df.groupby('article_id').agg({'relevant':'first',
                                                               'user_id':'first',
                                                               'locations':'first',
                                                               'sentiment':'first',
                                                               'usercoding_id':'first',
                                                               'category_id':', '.join}).reset_index()
print(rel_usercoded_df.shape)
display(rel_usercoded_df.head())

(510, 7)


,article_id,relevant,user_id,locations,sentiment,usercoding_id,category_id
0,42,t,NaN,[],NaN,42,SEXA
1,145,t,NaN,[],NaN,145,SEXA
2,205,t,NaN,[],NaN,205,SEXA
3,965,t,NaN,[],NaN,965,DOMV
4,1070,t,NaN,[],NaN,1070,DOMV


Merge news source names into articles DF

In [7]:
articles_merge_df = pd.merge(left=articles_df,
                      right=newssource_df,
                      left_on='news_source_id',
                      right_on='source_id')

In [8]:
# Can drop columns from newsource DF that are no longer needed
articles_merge_df = articles_merge_df.drop(['feedname',
                                            'source_id',
                                            'legacy_feed_id',
                                            'news_source_id',
                                            ], axis=1)

In [9]:
# Reorder DF so news source name is first after id
articles_merge_df = articles_merge_df[['id','source_name','short_name','url','title','bodytext',
                                       'relevant','created','last_modified','author']]

In [10]:
articles_user_merge_df = pd.merge(left=articles_merge_df,
                                 right=rel_usercoded_df,
                                 left_on='id',
                                 right_on='article_id')

In [11]:
print(articles_user_merge_df.shape)

(510, 17)


In [12]:
save = articles_user_merge_df[
    (pd.to_datetime(articles_user_merge_df["created"]) >= pd.to_datetime('2019-01-01 00:00:00.000000+0000')) &
    (pd.to_datetime(articles_user_merge_df["created"]) < pd.to_datetime('2020-01-01 00:00:00.000000+0000'))]
print(save.shape)
save.to_excel("/Users/mchladek/Downloads/usercoded_sexa_dv_85_2019.xlsx")

(14, 17)


TRAINED CODING

Rename columns so column names are unique after merge

In [13]:
trainedcoding_df.columns = ['id', 'date', 'model_info', 'overall_relevance',
                            'article_id', 'sentiment', 'bin', 'sentiment_processed']

In [14]:
trainedcategoryrelevance_df.columns = ['id','cat_relevance','category_id','coding_id']

Get trained codings with relevant crime categories

In [15]:
# Can drop sentiment related columns from trained coding; can drop 'id' column from categories
rel_trained_df = pd.merge(left=trainedcoding_df.drop(columns=['sentiment','bin','sentiment_processed']),
                         right=trainedcategoryrelevance_df.drop(columns=['id']),
                         left_on='id',
                         right_on='coding_id')
rel_trained_df = rel_trained_df[(rel_trained_df['cat_relevance'] >= 0.85) &
                               ((rel_trained_df['category_id'] == domv_cat) |
                               (rel_trained_df['category_id'] == sexa_cat))]

In [16]:
# Put each categories' relevance in a category-specific column
rel_trained_df.loc[rel_trained_df['category_id'] == domv_cat, 'category_id'] = 'DOMV'
rel_trained_df.loc[rel_trained_df['category_id'] == sexa_cat, 'category_id'] = 'SEXA'
rel_trained_df['sexa_relevance'] = np.where(rel_trained_df['category_id'].isin(['SEXA']), rel_trained_df['cat_relevance'], np.nan)
rel_trained_df['domv_relevance'] = np.where(rel_trained_df['category_id'].isin(['DOMV']), rel_trained_df['cat_relevance'], np.nan)

In [17]:
# Can drop category id's and relevance category now that transformed into their own columns
rel_trained_df = rel_trained_df.groupby('article_id').agg({'model_info':'first',
                                                           'overall_relevance':'first',
                                                           'coding_id':'first',
                                                           'sexa_relevance':'max',
                                                           'domv_relevance':'max'}).reset_index()
print(rel_trained_df.shape)
display(rel_trained_df.head())

(445, 6)


,article_id,model_info,overall_relevance,coding_id,sexa_relevance,domv_relevance
0,23181,tagnews 1.2.4,0.900334,1609121,0.894120,NaN
1,23352,tagnews 1.2.4,0.989070,1609292,NaN,0.98907
2,76369,tagnews 1.2.4,0.920908,1662312,0.920908,NaN
3,76814,tagnews 1.2.4,0.999996,1662757,0.999996,NaN
4,81333,tagnews 1.2.4,0.999989,1667276,0.999989,NaN


In [18]:
articles_trained_merge_df = pd.merge(left=articles_merge_df,
                                     right=rel_trained_df,
                                     left_on='id',
                                     right_on='article_id')
print(articles_trained_merge_df.shape)

(445, 16)


In [19]:
save = articles_trained_merge_df[
    (pd.to_datetime(articles_trained_merge_df["created"]) >= pd.to_datetime('2019-01-01 00:00:00.000000+0000')) &
    (pd.to_datetime(articles_trained_merge_df["created"]) < pd.to_datetime('2020-01-01 00:00:00.000000+0000'))]
print(save.shape)
save.to_excel("/Users/mchladek/Downloads/traincoded_sexa_dv_85_2019.xlsx")

(136, 16)
